# Projet télédétection
Audrey Zimmer

## 1. Import des librairies

In [37]:
# Librairies python
import sys
sys.path.append('..')
import os
from sklearn.model_selection import train_test_split
import numpy as np
import plotly.express as px
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, \
    accuracy_score
import geopandas as gpd
from osgeo import ogr
from osgeo import gdal
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold, KFold


# Librairies personnelles
from libsigma import classification as cla
from libsigma import read_and_write as rw
from libsigma import plots


## Création des dossiers

In [9]:
# Chemin de base (absolu)
my_folder = "/home/onyxia/work/Projet_teledetection"

# Chemins des dossiers à créer (absolus)
results_path = os.path.join(my_folder, "../results")
figure_path = os.path.join(my_folder, "../results/figure")
img_path = os.path.join(my_folder, "img")

# Création des dossiers
os.makedirs(results_path, exist_ok=True)
os.makedirs(figure_path, exist_ok=True)
os.makedirs(img_path, exist_ok=True)

print("Dossiers results, figure et img créés avec succès !")


Dossiers results, figure et img créés avec succès !


## Formatage des données

In [10]:
# Liste des bandes
bandes = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']

# Dictionnaire pour stocker les datasets
datasets = {}

# Vérification des dimensions
for bande in bandes:
    chemin = f'/home/onyxia/work/data/projet_eval/bretagne_23-24_{bande}.tif'
    datasets[bande] = gdal.Open(chemin, gdal.GA_ReadOnly)
    print(f"Bande {bande}:")
    print(f"  - Colonnes: {datasets[bande].RasterXSize}")
    print(f"  - Lignes: {datasets[bande].RasterYSize}")
    print(f"  - Nombre de bandes: {datasets[bande].RasterCount}\n")


Bande B02:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B03:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B04:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B05:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B06:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B07:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B08:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B8A:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B11:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B12:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5



Toutes les bandes obtenues ont la même dimension : 
- 1533 colonnes
- 612 lignes
- 5 bandes

In [11]:
# Liste des bandes
bandes = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']

# Liste des dates 
dates = ['2024-07-19', '2024-06-07', '2024-04-18', '2024-01-19', '2023-10-08']

# Dictionnaire pour stocker les datasets
datasets = {}

# Chargement des datasets
for bande in bandes:
    chemin = f'/home/onyxia/work/data/projet_eval/bretagne_23-24_{bande}.tif'
    datasets[bande] = gdal.Open(chemin, gdal.GA_ReadOnly)

# Dimensions spatiales
cols = datasets['B02'].RasterXSize
rows = datasets['B02'].RasterYSize

# Pour chaque date (indexé de 1 à 5)
for date_idx in range(1, len(dates) + 1):
    date = dates[date_idx - 1]  # Récupère la date correspondante
    # Tableau 3D pour stocker toutes les bandes pour cette date
    cube_date = np.zeros((rows, cols, len(bandes)), dtype=np.float32)

    # Pour chaque bande
    for i, bande in enumerate(bandes):
        # Lecture de la bande pour cette date
        data = datasets[bande].GetRasterBand(date_idx).ReadAsArray()
        cube_date[:, :, i] = data

    # Création du fichier de sortie (avec la date dans le nom)
    driver = gdal.GetDriverByName('GTiff')
    chemin_sortie = f'/home/onyxia/work/results/bretagne_{date}.tif'
    out_dataset = driver.Create(chemin_sortie, cols, rows, len(bandes), gdal.GDT_Float32)

    # Copie des informations géoréférencées
    out_dataset.SetGeoTransform(datasets['B02'].GetGeoTransform())
    out_dataset.SetProjection(datasets['B02'].GetProjection())

    # Écriture des bandes
    for i in range(len(bandes)):
        out_band = out_dataset.GetRasterBand(i + 1)
        out_band.WriteArray(cube_date[:, :, i])
        out_band.FlushCache()

    print(f"Image pour la date {date} créée : {chemin_sortie}")

Image pour la date 2024-07-19 créée : /home/onyxia/work/results/bretagne_2024-07-19.tif
Image pour la date 2024-06-07 créée : /home/onyxia/work/results/bretagne_2024-06-07.tif
Image pour la date 2024-04-18 créée : /home/onyxia/work/results/bretagne_2024-04-18.tif
Image pour la date 2024-01-19 créée : /home/onyxia/work/results/bretagne_2024-01-19.tif
Image pour la date 2023-10-08 créée : /home/onyxia/work/results/bretagne_2023-10-08.tif


## Analyse des échantillons
### Nombre d'échantillons

In [25]:
# Chemin vers le fichier shapefile
shp_path = '/home/onyxia/work/data/projet_eval/PI_strates_bretagne_32630.shp'

# Charger le fichier
gdf = gpd.read_file(shp_path)

# Compter le nombre de polygones par classe
polygon_counts = gdf['strate'].value_counts().reset_index()
polygon_counts.columns = ['Classe', 'Nombre de polygones']

# Convertir 'Classe' en chaîne de caractères pour éviter les conflits de type
polygon_counts['Classe'] = polygon_counts['Classe'].astype(str)

# Créer un DataFrame avec toutes les classes (1, 2, 3, 4), même si elles sont absentes
all_classes = pd.DataFrame({'Classe': ['1', '2', '3', '4']})

# Fusionner avec les comptes réels
polygon_counts = all_classes.merge(polygon_counts, on='Classe', how='left').fillna(0)

# Dictionnaire de correspondance classe/couleur (personnalisable)
color_map = {
    '1': '#F5DEB3',  # Sol nu (tan)
    '2': '#98FB98',  # Herbe (palegreen)
    '3': '#32CD32',  # Landes (limegreen)
    '4': '#228B22'   # Arbre (darkgreen)
}

# Créer le diagramme avec Plotly
fig_polygons = px.bar(
    polygon_counts,
    x='Classe',
    y='Nombre de polygones',
    color='Classe',
    color_discrete_map=color_map,
    title='Nombre de polygones par classe',
    labels={'Classe': 'Classe', 'Nombre de polygones': 'Nombre de polygones'},
    category_orders={'Classe': ['1', '2', '3', '4']}
)

# Personnaliser l'affichage des classes sur l'axe x
fig_polygons.update_xaxes(
    type='category',  # Forcer l'affichage en catégories discrètes
    tickvals=['1', '2', '3', '4'],  # Valeurs exactes à afficher
    ticktext=['Sol nu', 'Herbe', 'Landes', 'Arbre']   # Libellés à afficher
)

# Enregistrer la figure
output_polygon_path = '/home/onyxia/work/results/figure/diag_baton_nb_poly_by_class.html'
os.makedirs(os.path.dirname(output_polygon_path), exist_ok=True)
fig_polygons.write_html(output_polygon_path)

# Afficher la figure dans le notebook
fig_polygons.show()


La classe 1 (sol nu) est absente des 

### Nombre de pixels par classe

In [26]:
# Charger le shapefile
gdf = gpd.read_file('/home/onyxia/work/data/projet_eval/PI_strates_bretagne_32630.shp')

# Vérifier les valeurs uniques de 'strate'
print("Valeurs uniques de 'strate' :", gdf['strate'].unique())

# Vérifier le nombre de polygones par classe
print("Nombre de polygones par classe :\n", gdf['strate'].value_counts())


Valeurs uniques de 'strate' : [3 2 4]
Nombre de polygones par classe :
 strate
3    121
2     86
4     71
Name: count, dtype: int64


In [27]:
# Rasterisation
# Chemins des fichiers
in_vector = '/home/onyxia/work/data/projet_eval/PI_strates_bretagne_32630.shp'
out_image = '/home/onyxia/work/results/PI_strates_bretagne_32630_raster.tif'

# Étendue des polygones
xmin, ymin, xmax, ymax = 433831.49870281, 5362866.42545753, 446289.95490494, 5367061.39833999

# Résolution spatiale (10 mètres pour Sentinel-2)
resolution = 10

# Champ contenant les labels des classes
field_name = 'strate'

# Commande pour rasteriser
cmd = (
    f"gdal_rasterize -a {field_name} "
    f"-tr {resolution} {resolution} "
    f"-te {xmin} {ymin} {xmax} {ymax} "
    "-ot Byte -of GTiff -init 0 "
    f"{in_vector} {out_image}"
)

print(f"Commande exécutée : {cmd}")
os.system(cmd)


Commande exécutée : gdal_rasterize -a strate -tr 10 10 -te 433831.49870281 5362866.42545753 446289.95490494 5367061.39833999 -ot Byte -of GTiff -init 0 /home/onyxia/work/data/projet_eval/PI_strates_bretagne_32630.shp /home/onyxia/work/results/PI_strates_bretagne_32630_raster.tif
0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [28]:
# Vérification des valeurs présentes dans le raster
# Lire le raster rasterisé
raster_ds = gdal.Open(out_image)
band = raster_ds.GetRasterBand(1)
raster_array = band.ReadAsArray()

# Afficher les valeurs uniques et leurs comptes
unique, counts = np.unique(raster_array, return_counts=True)
print("Valeurs uniques dans le raster :", dict(zip(unique, counts)))


Valeurs uniques dans le raster : {np.uint8(0): np.int64(518226), np.uint8(2): np.int64(1044), np.uint8(3): np.int64(1575), np.uint8(4): np.int64(1229)}


In [31]:
# Compter les pixels par classe
unique, counts = np.unique(raster_array, return_counts=True)
pixel_counts = dict(zip(unique, counts))

# Exclure la classe 0 (fond)
pixel_counts.pop(0, None)

# Créer un DataFrame avec toutes les classes (1, 2, 3, 4)
all_classes = pd.DataFrame({'Classe': [1, 2, 3, 4]})

# Fusionner avec les comptes de pixels
pixel_df = pd.merge(
    all_classes,
    pd.DataFrame({'Classe': list(pixel_counts.keys()), 'Nombre de pixels': list(pixel_counts.values())}),
    on='Classe',
    how='left'
).fillna(0)

# Dictionnaire de couleurs
color_map = {
    1: '#F5DEB3',  # Sol nu
    2: '#98FB98',  # Herbe
    3: '#32CD32',  # Landes
    4: '#228B22'   # Arbre
}

# Créer le diagramme
fig_pixels = px.bar(
    pixel_df,
    x='Classe',
    y='Nombre de pixels',
    color='Classe',
    color_discrete_map=color_map,
    title='Nombre de pixels par classe',
    labels={'Classe': 'Classe', 'Nombre de pixels': 'Nombre de pixels'},
    category_orders={'Classe': [1, 2, 3, 4]}
)

# Personnaliser l'axe x
fig_pixels.update_xaxes(
    type='category',
    tickvals=[1, 2, 3, 4],
    ticktext=['1', '2', '3', '4']
)

# Enregistrer la figure
output_pixel_path = '/home/onyxia/work/results/figure/diag_baton_nb_pix_by_class.html'
os.makedirs(os.path.dirname(output_pixel_path), exist_ok=True)
fig_pixels.write_html(output_pixel_path)

# Afficher la figure
fig_pixels.show()


In [34]:
import plotly.graph_objects as go

# Créer le diagramme en utilisant plotly.graph_objects
fig_pixels = go.Figure()

# Ajouter les barres avec les couleurs exactes
for classe in pixel_df['Classe']:
    count = pixel_df[pixel_df['Classe'] == classe]['Nombre de pixels'].values[0]
    fig_pixels.add_trace(go.Bar(
        x=[classe],
        y=[count],
        marker_color=color_map[classe],
        name=str(classe)
    ))

# Mettre à jour la mise en page
fig_pixels.update_layout(
    title='Nombre de pixels par classe',
    xaxis_title='Classe',
    yaxis_title='Nombre de pixels',
    xaxis=dict(
        tickvals=[1, 2, 3, 4],
        ticktext=['Sol nu', 'Herbe', 'Landes', 'Arbre'],
        type='category'
    ),
    showlegend=False
)

# Afficher la figure
fig_pixels.show()

# Enregistrer la figure
output_pixel_path = '/home/onyxia/work/results/figure/diag_baton_nb_pix_by_class.html'
os.makedirs(os.path.dirname(output_pixel_path), exist_ok=True)
fig_pixels.write_html(output_pixel_path)


## Phénologie des strates, mise en évidence des landes

In [38]:


# Chemins des fichiers d'entrée et de sortie
input_folder = '/home/onyxia/work/results/'
output_folder = '/home/onyxia/work/results/'
output_file = os.path.join(output_folder, 'ARI_serie_temp.tif')

# Liste des dates (adapte selon tes fichiers)
dates = ['2023-10-08', '2024-01-19', '2024-04-18', '2024-06-07', '2024-07-19']

# Lire les dimensions et la projection depuis une image de référence
ref_image_path = os.path.join(input_folder, 'bretagne_2023-10-08.tif')
ref_ds = gdal.Open(ref_image_path)
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
transform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()

# Créer un fichier de sortie pour ARI
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_file, cols, rows, len(dates), gdal.GDT_Float32)
out_ds.SetGeoTransform(transform)
out_ds.SetProjection(projection)

# Définir la valeur NoData
for band in range(1, len(dates) + 1):
    out_band = out_ds.GetRasterBand(band)
    out_band.SetNoDataValue(-9999)

# Calculer ARI pour chaque date
for i, date in enumerate(dates):
    # Chemins des bandes B03 et B05
    b03_path = os.path.join(input_folder, f'bretagne_{date}.tif')
    b05_path = os.path.join(input_folder, f'bretagne_{date}.tif')

    # Ouvrir les bandes B03 et B05
    b03_ds = gdal.Open(b03_path)
    b05_ds = gdal.Open(b05_path)

    # Lire les bandes 3 et 5 (adapte les indices selon ton organisation)
    b03_band = b03_ds.GetRasterBand(3)  # Supposons que B03 est la bande 3
    b05_band = b05_ds.GetRasterBand(5)  # Supposons que B05 est la bande 5

    # Lire les données
    b03 = b03_band.ReadAsArray().astype(np.float32)
    b05 = b05_band.ReadAsArray().astype(np.float32)

    # Éviter les divisions par zéro en ajoutant un epsilon
    epsilon = 1e-10
    ari = np.where((b03 == 0) | (b05 == 0), np.nan, (1/(b03 + epsilon) - 1/(b05 + epsilon)) / (1/(b03 + epsilon) + 1/(b05 + epsilon)))

    # Remplacer les NaN par la valeur NoData (-9999)
    ari = np.where(np.isnan(ari), -9999, ari)

    # Écrire ARI dans la bande de sortie
    out_band = out_ds.GetRasterBand(i + 1)
    out_band.WriteArray(ari)
    out_band.FlushCache()

# Fermer les fichiers
out_ds = None

print(f"Fichier ARI_serie_temp.tif enregistré avec succès dans {output_folder}")


Fichier ARI_serie_temp.tif enregistré avec succès dans /home/onyxia/work/results/
